In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg-packages'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/usr/lib/ultralytics_for_offline_install_mine/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [ ]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_model
import glob

In [ ]:
# Construct model
model = flg_model.FinalModel()
model.ratio_of_motors_allowed = 0.45
model.estimate_voxel_spacing = True
model.run_in_parallel = True
model.state = 1
if not fls.env=='kaggle':    
    res_dir = fls.model_dir + '/many_full_res/'
    #fls.download_kaggle_dataset('jeroencottaar/byu-many-models/', res_dir)
else:
    res_dir = '/kaggle/input/byu-many-models/'
files = glob.glob(res_dir+'/*.pickle')
files.sort()
for f in files:
    data = fls.dill_load(f)
    this_model = data.trained_model
    this_model.step1Labels.preprocessor.voxel_scale = 1.
    model.step1_list.append(this_model.step1Labels)

In [ ]:
# Infer
inferred_test_data = model.infer(fls.load_all_test_data())
for d in inferred_test_data:
    print(d.name)
    print(d.labels)
fls.write_submission_file(inferred_test_data)

In [ ]:
%%time
if not fls.is_submission:
    data = model.infer(fls.load_all_train_data()[::10])
    for d in data:
        print(d.labels)